# Assessing Player Value through Quantitative Measures of Match Performance

## CS 109 Final Project by FIFA (Future Impact Football Analytics)
##### Presented by Taylor Killian, Virgile Audi and Abhishek Malali

#Table of Contents
* [Assessing Player Value through Quantitative Measures of Match Performance](#Asessing-Player-Value-through-Quantitative-Measures-of-Match-Performance)
	* [Section 0: Project Overview](#Section 0:-Project-Overview)
		* [Background and Motivation](#Background-and-Motivation)
		* [Project Objectives](#Project-Objectives)
	* [Section 1: Gathering Data: Acquisition and Cleaning](#Section 1:-Gathering-Data:-Acquisition-and-Cleaning)
		* [1.1: Scraping Data](#1.1:-Scraping-Data)
			* [Match Statistics](#Match-Statistics)
			* [Transfer Values](#Transfer Values)
		* [1.2: Data Cleaning](#1.2:-Data-Cleaning)
            * [Calculation of Player Impact](#Calculation-of-Player-Impact)
	* [Section 2: Feature Extraction and ML](#Section-2:-Feature-Extraction-and-ML)
        * [2.1: Continued Data Munging](#2.1:-Continued-Data-Munging)
		* [2.2: EDA via Methods of ML](#2.2: EDA-via-Methods-of-ML)
            * [Validating our Metric](#Validating-out-Metric)
            * [Clustering](#Clustering)
	* [Section 3: Further Analysis and Visualization](#Section 3:Further-Analysis-and-Visualization)
    * [Section 4: Conclusions](#Section 4:-Conclusions)


In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

## Section 0: Project Overview

### Background and Motivation

The three of us are large football (and by football, we of course mean the one and only, the one played with feet) fans. We consume the game in many aspects of our lives. Our hearts are lifted when our respective teams do well and are similarly frustrated when they fail to perform to our expectations. When teams do not perform equal to their, and their fans', expectations they look to infuse their team with more talented players, typically drawn from lower-tier or financially poor teams. The financial incentive for both teams in any transaction is clear, the buying team has hope that the new addition will put them over the top talent-wise to improve their standing in league competition while the selling team may look at the new funds as a way to prepare for the future and secure several new players to improve their team long-term.

In the past few years, the economy of player transfers has undergone significant inflation where the top-tier clubs have shown a willingness to overpay for marginal talent or for players that may not be a good fit in their team. There is some notion that the transfer system is broken and is in need of revitalization. There are multiple ways to address the psychology of club executives and try to identify their motivations for spending money to procure new talent for their team. We refrain from any investigation into club finances but want to determine a method by which we can accurately and objectively place value on any player based on the impact that they have on the matches they play in.

### Project Objectives

Our primary motivation is to quantify the impact (based on goals scored, assists made and consistency of play) an individual player has on the games which he plays for his team. We intend to use this quantity for each player to infer their value (agnostic of team at first and later based on current team and potential buyers). We hope to learn more about what goes into modeling player value, beyond subjective measures of team they currently play for and the league they currently play in, and how we can compare like players with each other. Our hope is that this work will provide the foundation to build an analysis portfolio to enable application to sports analytics positions or, at the very least, a long-term hobby.

## Section 1: Gathering Data: Acquisition and Cleaning

As the use of statistical analysis within sport has [matured and been formalized](http://www.forbes.com/sites/leighsteinberg/2015/08/18/changing-the-game-the-rise-of-sports-analytics/), there have appeared a variety of organizations (some public, most private) whose sole purpose is to record and store event data for use in developing a greater understanding of sport analytically. In baseball, this was done (at least initially) by amateurs and news organizations as a way to better describe and follow the game as well as player performance. This practice has evolved to a point where most professional teams in baseball employ full-time analysts employed to delve into intracacies of player statistics in order to find any competitive edge. Over time, more sports and leagues have begun to rely on quantitative measures to influence organizational as well as gameplay decisions. This focus on analytics has ushered in a wave of private companies who track and record minutae of each game or match with the design to sell this information to the organizations involved with sport.

Our team (FIFA) hoped to get a hold of some of this data but found the subscription cost to statistical sources with finer detail to be prohibitive for a final project. Since we would not have access to the most detailed of data we were forced to redirect our analytical plan to only include match results (Teams, goals, assists, time of important events and players involved). We scoured multiple websites in search of historical data from the top 5 European football (soccer) leagues for the 2012-2013 and 2013-2014 seasons. After a few false starts and dead-ends, we were fortunate to find [www.worldfootball.net](http://www.worldfootball.net) that contained exactly the data we were looking for.

### 1.1: Scraping Data


#### Match Statistics

Significant effort was required to understand how worldfootball.net encoded their match result webpages. Once that was understood, the html from each match played over the course of the two seasons was parsed and summarized into a .json file. Each .json recorded the names of the teams that faced off, the goals and assists recorded by each team, the players involved in each score, what time they occured and also some second order features of what effect that goal had on the match (if it broke a tie, was an equalizer, etc).

We updated the scraping function iteratively as we determined deeper aspects that influenced a players impact on a match and found better ways to organize and compare our data. Two such updates were including whether a player was a substitute (if a substitute can positively effect the outcome of a match, he has to be given some added bonus to his value score) and what point in the season the match was played (helped to organize and unify the dataset).

Our scraping function and process can be viewed at [scraping.ipynb](Scraping/scraping.ipynb)

#### Transfer Values

We wanted to see if there was a correlation between our metric and how players have been valued over the course of the years we recorded. This was a fairly limited data set as many transfers are undisclosed and the number of transfers completed from one season to the next is quite small when compared with the total number of players. 

The work done along that vein can be seen in [Transfer_values_scraping.ipynb](Scraping/Transfer_values_scraping.ipynb)

### 1.2 Data Cleaning

In order to efficiently scrape the match data from www.worldfootball.net a good deal of cleaning was already performed. However, since we were focused on individual player performance, there was a need to parse the match data and clean it up in order to provide a database of players, including the contributions each player made on a particular match day. All information used to calculate player impact, according to our metrics, was stored in these databases following cleaning.

Link to [Init_DataCleaning.ipynb](Data Cleaning and Feature Extraction/Init_DataCleaning.ipynb)

#### Calculation of Player Impact

Since we use are using goals and assists to define a player's impact (by virtue of the data we had access to within the time constraints of our project), it's abundantly clear that our databases are biased toward offensively minded players. To differentiate between these players, we tried to identify which match states and types of goals are more impactful for a team during a game. We weighted these kinds of goals and assists more heavily which would provide for a better score for a player. 

We split each 90-min match into three time intervals: Early ($\leq$ 20 min), Mid ( > 20 min and < 80 min) and Late ($\geq$ 80 min). We also split the goals/assists into three categories; tiebreakers, equalizers and nominal. Each of these kinds of goals/assists were weighted differently depending on when in the match they were recorded.

As mentioned previously, players were given a bonus if they recorded a statistic after coming into the match as a substitute. Another bonus that we provided players was if they recorded a positive statistic (we, as a rule to simplify our analysis and not dwell on extremely rare events, ignore own-goals) if they were on the "away" team. It is well understood and intuitive that it is more difficult in football to score away from home, we chose to reflect that in our analysis by a small increment. We also determined that goals are more valuable than assists, so the two were weighted differently (but only *slightly*) for each contributing player.

We also chose to reward players for consistency. We view a player as more valuable if they score a few goals frequently rather than a player who may score a lot at one time but is otherwise absent from the scoresheet. To do this, we created an incremental increase to the weight for a goal or assist if the player registered a postive statistic in the previous match.

To track a player's impact over the course of the season we also record a player's cumulative value for each match day. We also wanted to smooth for inconsistency in this feature by tracking a smoothed average of player impact over the previous five matches.

Further details and information can be found in [initial_value_calculations.ipynb](Data Cleaning and Feature Extraction/initial_value_calculations.ipynb)

## Section 2: Feature Extraction and ML

### 2.1: Continued Data Munging

Link to [continued_data_munging.ipynb](Data Cleaning and Feature Extraction/continued_data_munging.ipynb)

### 2.2: EDA via Methods of ML

#### Validating our Metric

Link to [ML_Regression.ipynb](Analysis/ML_Regression.ipynb)

#### Clustering

Link to [clustering-test-notebook.ipynb](Analysis/clustering-test-notebook.ipynb)

## Section 3: Further Analysis and Visualization

Link to [Match_predictions.ipynb](Analysis/Match_predictions.ipynb)

Link to [finding_percentiles.ipynb](Analysis/finding_percentiles.ipynb)

Link to [transfer-data-test.ipynb](Plots/transfer-data-test.ipynb)

Link to [Plotting.ipynb](Plots/Plotting.ipynb)

## Section 4: Conclusions